# Digital House - Data Science a Distancia

## Trabajo Práctico 2

Prepara el dataset original con las características que se presentan en el [valuador de Properati](https://www.properati.com.ar/tools/valuador-propiedades)

### Autores: Daniel Borrino, Ivan Mongi, Jessica Polakoff, Julio Tentor

<p style="text-align:right;">Mayo 2022</p>


#### Aspectos técnicos
La notebook se ejecuta correctamente en una instalación estándar de Anaconda versión 4.11.0 build  3.21.6, Python 3.9.7


#### Librerías necesarias

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import matplotlib.pyplot as plt

In [2]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [3]:
data_url = "../Data/properatti.csv"
data = pd.read_csv(data_url, encoding="utf-8")

---
### Generación del dataset final

#### Eliminamos los valores nulos de la variable Target

In [4]:
data.rename(columns={'property_type' : 'tipo', 'price_aprox_usd' : 'precio','surface_covered_in_m2' : 'sup'}, inplace=True)

In [5]:
#Limpiamos los NaN en el precio
data = data.dropna(axis=0, how='any', subset=['precio', 'sup'])
data_clean = data

#### Seleccionamos solo Capital Federal y Bs.As. zonas Norte, Sur y Oeste

In [6]:
#Seleccionamos solo Capital Federal y Bs.As. zonas Norte, Sur y Oeste
# iterar_state = ['Capital Federal',
#                 'Bs.As. G.B.A. Zona Norte',
#                 'Bs.As. G.B.A. Zona Sur',
#                 'Bs.As. G.B.A. Zona Oeste']

iterar_state = data_clean['state_name'].value_counts().head(6)
iterar_state = iterar_state.index

data_clean['state_name'] = [x if x in iterar_state else np.NaN for x in data_clean['state_name']]
data_clean = data_clean.dropna(axis=0, how='any', subset=['state_name']).copy()

#### Seleccionamos solo Departamento, Casa y PH

In [7]:
#Seleccionamos solo Departamento, Casa y PH
iterar_tipo = data_clean['tipo'].value_counts().head(3)
iterar_tipo = iterar_tipo.index

data_clean['tipo'] = [x if x in iterar_tipo else np.NaN for x in data_clean['tipo']]
data_clean = data_clean.dropna(axis=0, how='any', subset=['tipo']).copy()

In [8]:
data_clean['place_name'].value_counts().head(100)

Córdoba            4719
Mar del Plata      4226
Tigre              2949
Nordelta           2785
Belgrano           2362
                   ... 
Burzaco             155
Valentín Alsina     154
El Palomar          152
Muñiz               151
Ciudadela           148
Name: place_name, Length: 100, dtype: int64

#### Seleccionamos solo Lugares con muchas observaciones

In [9]:
#Seleccionamos solo Lugares con muchas observaciones
iterar_place = data_clean['place_name'].value_counts()[:100]
iterar_place = iterar_place.index

data_clean['place_name'] = [x if x in iterar_place else 'otro' for x in data_clean['place_name']]
data_clean = data_clean.dropna(axis=0, how='any', subset=['place_name']).copy()

#### Eliminamos Outliers para las variables que vamos a correlacionar: 

In [10]:
#funcion para borrar outliers.
def borrar_outliers(data, columnas):
    u"""Solo recibe columnas con valores numericos. 
    Data: dataset a analizar
    Columnas: columnas donde borrar outliers.Deben ser una tupla"""
    cols_limpiar = columnas
    mask=np.ones(shape=(data.shape[0]), dtype=bool)

    for i in cols_limpiar:
        
        #calculamos cuartiles, y valores de corte
        Q1=data[i].quantile(0.25)
        Q3=data[i].quantile(0.75)
        RSI=Q3-Q1
        max_value=Q3+1.5*RSI
        min_value=Q1-1.5*RSI
        
        #ajusto el min value 
            # No puede ser negativo.
            # No puede estar fuera del boxplot para outliers
            # Criterio experto se decide dejar desde el 5% hacia adelante en el precio.
            # Además, no consideraremos los que tienen menos de 10m2.
        if i=='precio':
            min_value=max(data[i].quantile(0.05), min_value, 1000)
        else: 
            min_value=max(data[i].quantile(0.05), min_value, 10)
        
        max_value=(min(data[i].quantile(0.85), max_value, 800000))
        
        #filtramos por max y min
        mask=np.logical_and(mask, np.logical_and(data[i]>=min_value, data[i]<=max_value))
    return data[mask]

In [11]:
# serie para determinar observaciones sin outliers para precio en dólares y superficie cubierta
data_clean['tidy1'] = np.NaN

for tipo in iterar_tipo:
    for place in iterar_place:
        # selecciono por lugar y tipo
        mask = np.logical_and(data_clean['place_name']==place, data_clean['tipo']==tipo)
        # calcula outliers y los suprime
        data_ok = borrar_outliers(data_clean[mask], ('precio', 'sup'))
        # determina observaciones válidas
        data_clean.loc[data_ok.index, 'tidy1'] = True

# probablemente esto se pueda hacer con algún método de pandas pero me resulta más simple pensarlo de este modo

In [12]:
# suprimo las observaciones que no me sirven
data_clean = data_clean.dropna(axis=0, how='any', subset=['tidy1'])

---
#### Creacion de nuevas variables con valor predictivo:


##### Analisis para Cantidad de ambientes

In [13]:
def regex_to_values(col, reg, not_match=0) :
    u"""Returns a serie with the result of apply the regular expresion to the column
    the serie have a float value only when regular expression search() method found a match
    
    col : column where to apply regular expresion
    reg : regular expresion compiled
    """
    
    serie = col.apply(lambda x : x if x is np.NaN else reg.search(x))
    serie = serie.apply(lambda x : not_match if x is np.NaN or x is None else float(x.group(1)))

    return serie

In [14]:
#Buscamos cantidad de ambientes
_pattern = '([1-2][0-9]?)(?= amb)'
_express = re.compile(_pattern, flags = re.IGNORECASE)

work = regex_to_values(data_clean['description'], _express, 1)

data_clean['ambientes'] = work


In [15]:
#realizamos la imputacion
#data_clean['ambientes_final'] = data_clean['rooms']
#mask = data_clean['ambientes_final'].isnull()
#data_clean.loc[mask, 'ambientes_final'] = data_clean.loc[mask, 'ambientes']

mask = data_clean['rooms'].notnull()
data_clean.loc[mask, 'ambientes'] = data_clean.loc[mask, 'rooms']

##### Analisis para Cantidad de baños

In [16]:
_pattern = '([1-2][0-9]?)(?= baño)'
_express = re.compile(_pattern, flags = re.IGNORECASE)

work = regex_to_values(data['description'], _express, 1)

data_clean['baños'] = work


---
##### Nos proponemos encontrar amenities

In [17]:
def regex_to_tags(col, reg, match, not_match = np.NaN) :
    u"""Returns a series with 'match' values result of apply the regular expresion to the column
    the 'match' value will be when the regular expression search() method found a match
    the 'not_match' value will be when the regular expression serach() method did not found a match
    col : column where to apply regular expresion
    reg : regular expresion compiled
    """
    
    serie = col.apply(lambda x : x if x is np.NaN else reg.search(x))
    serie = serie.apply(lambda x : match if x is not None else not_match)
   
    return serie

In [18]:
#Buscamos Balcón
_pattern = 'balcon|balcón'
_express = re.compile(_pattern, flags = re.IGNORECASE)

data_clean['balcon'] = regex_to_tags(data_clean['description'], _express, 1, 0)


In [19]:
#Buscamos Cocheras
_pattern = 'cochera|garage|auto'
_express = re.compile(_pattern, flags = re.IGNORECASE)

data_clean['cochera'] = regex_to_tags(data_clean['description'], _express, 1, 0)


In [20]:
#Buscamos Parrillas
_pattern = 'parrilla'
_express = re.compile(_pattern, flags = re.IGNORECASE)

data_clean['parrilla'] = regex_to_tags(data_clean['description'], _express, 1, 0)


In [21]:
#Buscamos Piletas
_pattern = 'piscina|pileta'
_express = re.compile(_pattern, flags = re.IGNORECASE)

data_clean['pileta'] = regex_to_tags(data_clean['description'], _express, 1, 0)



In [22]:
#Buscamos Amoblado
_pattern = 'amoblado'
_express = re.compile(_pattern, flags = re.IGNORECASE)

data_clean['amoblado'] = regex_to_tags(data_clean['description'], _express, 1, 0)


In [23]:
#Buscamos Lavadero
_pattern = 'lavadero'
_express = re.compile(_pattern, flags = re.IGNORECASE)

data_clean['lavadero'] = regex_to_tags(data_clean['description'], _express, 1, 0)


In [24]:
#Buscamos Patio
_pattern = 'patio'
_express = re.compile(_pattern, flags = re.IGNORECASE)

data_clean['patio'] = regex_to_tags(data_clean['description'], _express, 1, 0)


In [25]:
#Buscamos Terraza
_pattern = 'terraza'
_express = re.compile(_pattern, flags = re.IGNORECASE)

data_clean['terraza'] = regex_to_tags(data_clean['description'], _express, 1, 0)


In [26]:
#Buscamos Jardin
_pattern = 'jardin'
_express = re.compile(_pattern, flags = re.IGNORECASE)

data_clean['jardin'] = regex_to_tags(data_clean['description'], _express, 1, 0)


In [27]:
data_clean.reset_index(inplace=True)

In [28]:
data_clean = data_clean.drop(columns=['index', 'Unnamed: 0','operation', 'place_with_parent_names', 
                                      'country_name','geonames_id', 'lat-lon', 'lat', 'lon', 'price', 'currency',
                                        'price_aprox_local_currency','floor', 'price_usd_per_m2', 
                                     'price_per_m2', 'rooms', 'expenses', 'properati_url', 'description', 'title', 'image_thumbnail', 'surface_total_in_m2', 
                                      'tidy1'])

In [29]:
data_clean.columns

Index(['tipo', 'place_name', 'state_name', 'precio', 'sup', 'ambientes',
       'baños', 'balcon', 'cochera', 'parrilla', 'pileta', 'amoblado',
       'lavadero', 'patio', 'terraza', 'jardin'],
      dtype='object')

In [30]:
data_clean = pd.get_dummies(data=data_clean, columns=['tipo', 'place_name','state_name'], drop_first=True)
data_clean.columns

Index(['precio', 'sup', 'ambientes', 'baños', 'balcon', 'cochera', 'parrilla',
       'pileta', 'amoblado', 'lavadero',
       ...
       'place_name_Villa Luro', 'place_name_Villa Luzuriaga',
       'place_name_Villa Pueyrredón', 'place_name_Villa Urquiza',
       'place_name_Villa del Parque', 'state_name_Bs.As. G.B.A. Zona Oeste',
       'state_name_Bs.As. G.B.A. Zona Sur',
       'state_name_Buenos Aires Costa Atlántica', 'state_name_Capital Federal',
       'state_name_Córdoba'],
      dtype='object', length=119)

In [31]:
data_clean['sup2']=data_clean['sup']**2

In [32]:
# data_clean['supxhouse']=data_clean['sup']*data_clean['tipo_house']
# data_clean['supxapartment']=data_clean['sup']*data_clean['tipo_apartment']

amenities= ['balcon', 'cochera', 'parrilla', 'pileta', 'amoblado', 'lavadero', 'patio', 'terraza', 'jardin']

for a in amenities:
    cat1='tipo_house'+'x'+a
    data_clean[cat1]=data_clean['tipo_house']*data_clean[a]
    
    cat2='tipo_apartment'+'x'+a
    data_clean[cat2]=data_clean['tipo_apartment']*data_clean[a]

if True :    
    for b in data_clean.columns:
        if b=="precio":
            continue
        cat3='sup'+'x'+b
        data_clean[cat3]=data_clean['sup']*data_clean[b]

C:\Users\jtentor\AppData\Local\Temp/ipykernel_17168/3969809446.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data_clean[cat3]=data_clean['sup']*data_clean[b]


In [33]:
data_final=data_clean.copy()

In [34]:
data_final.columns

Index(['precio', 'sup', 'ambientes', 'baños', 'balcon', 'cochera', 'parrilla',
       'pileta', 'amoblado', 'lavadero',
       ...
       'supxtipo_housexamoblado', 'supxtipo_apartmentxamoblado',
       'supxtipo_housexlavadero', 'supxtipo_apartmentxlavadero',
       'supxtipo_housexpatio', 'supxtipo_apartmentxpatio',
       'supxtipo_housexterraza', 'supxtipo_apartmentxterraza',
       'supxtipo_housexjardin', 'supxtipo_apartmentxjardin'],
      dtype='object', length=275)

In [35]:
excluir=['precio']
features = data_final.columns.difference(excluir)

In [36]:
scaler = StandardScaler()

#data_final[features] = scaler.fit_transform(data_final[features])

In [37]:
data_final

,precio,sup,ambientes,baños,balcon,cochera,parrilla,pileta,amoblado,lavadero,...,supxtipo_housexamoblado,supxtipo_apartmentxamoblado,supxtipo_housexlavadero,supxtipo_apartmentxlavadero,supxtipo_housexpatio,supxtipo_apartmentxpatio,supxtipo_housexterraza,supxtipo_apartmentxterraza,supxtipo_housexjardin,supxtipo_apartmentxjardin
0,62000.0,40.0,2.0,1.0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,72000.0,55.0,2.0,1.0,0,0,0,0,0,1,...,0.0,0.0,0.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0
2,64000.0,35.0,1.0,1.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,130000.0,78.0,1.0,1.0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,138000.0,40.0,1.0,1.0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47507,128000.0,35.0,1.0,1.0,0,1,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0
47508,165000.0,39.0,1.0,1.0,1,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47509,498000.0,360.0,1.0,1.0,0,1,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,360.0,0.0
47510,131500.0,39.0,1.0,1.0,1,1,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,0.0,0.0


In [38]:
#--------------------------------------------------------------#

In [39]:
#divido en data train y data test

In [40]:
data_train, data_test = train_test_split(data_final, test_size=0.35, random_state =300)

In [41]:
target =data_train.precio

X = data_train[features]
y = target

In [42]:
# model = linear_model.Lasso(alpha=1, normalize=True)
# model = linear_model.ElasticNetCV(l1_ratio=[1], alphas=[0.000001], normalize=False, cv=3, max_iter=2000)
model = linear_model.LinearRegression()

#Probe un monton de combinaciones. No encontre overfit en ninguna. Vamos directamente con LinearRegression

In [43]:
#No entiendo porque, pero si llamo a model.fit varias veces, funciona. Si lo uso una sola vez no funciona (overfitea a morir)

In [44]:
model.fit(X, y)

LinearRegression()

In [45]:
##Checkeo de modelo

In [46]:
n_train=data_train.shape[0]
r2_train=model.score(X=data_train[features], y = data_train.precio)
r2_adj_train = 1-(1-r2_train)*((n_train-1)/(n_train-len(features)-1))
precio_est_train=model.predict(data_train[features])
RMSE_train= np.sqrt(mean_squared_error(y_true=data_train['precio'], y_pred=precio_est_train))
print(f'r2_train: {r2_train} \nr2_adj_train: {r2_adj_train} \nRMSE: {RMSE_train} ')

r2_train: 0.8093940357667988 
r2_adj_train: 0.8076876929628685 
RMSE: 53704.38611554713 


In [47]:
n_test=data_test.shape[0]
r2_test=model.score(X=data_test[features], y = data_test.precio)
r2_adj_test = 1-(1-r2_test)*((n_test-1)/(n_test-len(features)-1))
precio_est_test=model.predict(data_test[features])
RMSE_test= np.sqrt(mean_squared_error(y_true=data_test['precio'], y_pred=precio_est_test))
print(f'r2_test: {r2_test} \nr2_adj_test: {r2_adj_test} \nRMSE: {RMSE_test} ')

r2_test: 0.8152654594270593 
r2_adj_test: 0.8121705487503864 
RMSE: 54096.52603532372 


In [49]:
data_train['precio'].describe()

count     30882.000000
mean     180249.450816
std      123012.301850
min       36000.000000
25%       95000.000000
50%      139000.000000
75%      226371.250000
max      800000.000000
Name: precio, dtype: float64

In [50]:
data_test['precio'].describe()

count     16630.000000
mean     181923.851132
std      125865.985928
min       36000.000000
25%       95000.000000
50%      140000.000000
75%      229000.000000
max      800000.000000
Name: precio, dtype: float64